<a href="https://colab.research.google.com/github/roschkoenig/Automatic_EEG_Reporting/blob/main/Automatic_EEG_Reporting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget
!pip install openpyxl==3.0.9

import ipywidgets as widgets 
import pandas as pd
import gspread 
from google.colab import files
import wget 
import pandas as pd
import numpy as np
import os
try: os.remove('EEGReportingAutomation.xlsx') 
except: print('No file')

wget.download('https://docs.google.com/spreadsheets/d/e/2PACX-1vSu3hIdc_Pe_HW56hM3_fe941j9rykT8Bxv3N86T5_X5yh4WUnFKrZik7RPfl-Itgvh0MqI-ZLt14Dd/pub?output=xlsx')

global choices, vec2text, buttons
xlsfile  = pd.ExcelFile('EEGReportingAutomation.xlsx')
vec2text = pd.read_excel(xlsfile, None)

In [ ]:
choices = {}

# Define output widget (text box)
#-------------------------------------------------------------------------------
out = widgets.Textarea(
    value = '', 
    rows = 10,
    layout = widgets.Layout(
        min_width = '600px',
        )
    )


# Set up buttons object
#-------------------------------------------------------------------------------
sections = vec2text.keys()
buttons = []
for s in sections:
  buttons.append({'name': s, 'options': vec2text[s].columns[:-1]})

# Setting up options with multiple allowed answers 
#-------------------------------------------------------------------------------
def toggle_multiple(title, button_obj):
  input_words = button_obj['options']
  section     = button_obj['name']

  if section not in choices.keys(): choices[section] = pd.DataFrame()
  tbs = {}
  tbd = []

  for word in input_words:
    tbs[word] = widgets.ToggleButton(
        description = word, 
        tooltip = section, # using this to store button identity
        value = False
        )
    choices[section][word] = [tbs[word].value]
    tbs[word].observe(on_multitoggle, 'value')
    tbd.append(tbs[word])
    choices[section][word] = tbs[word].value
  tbd.insert(0, widgets.Label(title+': '))
  tbd = widgets.Box(
        tbd,
        layout = widgets.Layout(
          display = 'flex',
          flex_flow = 'col_wrap'
        )
      )
  return tbd

def toggle_single(title, button_obj):
  input_words = button_obj['options']
  section     = button_obj['name']
  if section not in choices.keys(): choices[section] = pd.DataFrame()

  tbs = widgets.ToggleButtons(
      options = input_words,
      description = title,
      tooltips = [section for word in input_words], # using this to store button identity
  )
  for word in input_words: choices[section][word] = [False]
  tbs.observe(on_singletoggle, 'value')
  
  return tbs

# Function called when input changes
#-------------------------------------------------------------------------------
def on_multitoggle(change):
  section = change['owner'].tooltip
  choices[section][change['owner'].description] = [change['owner'].value]
  out.value = translate()

def on_singletoggle(change):
  section = change['owner'].tooltips[0]
  choices[section][change['old']] = False
  choices[section][change['new']] = True
  out.value = translate()

# Takes 
#-------------------------------------------------------------------------------
def translate():
  sections    = choices.keys()
  bsecs       = [b['name'] for b in buttons]
  return_text = ''

  for sec in sections:
    options = buttons[bsecs.index(sec)]['options']
    df      = vec2text[sec][options].fillna(False).astype('int').sort_index()
    sample  = choices[sec][options].fillna(False).astype('int').sort_index()

    for r in range(df.shape[0]):  
      if df.loc[r].equals(sample.loc[0]):
        cellcont = vec2text[sec]['text'].loc[r]
        stringlet = cellcont if isinstance(cellcont, str) else '' 
        return_text = return_text + stringlet
  
  return return_text

In [ ]:
# state_options = ['awake', 'asleep', 'encephalopathic','sedated', 'comatose', 'indeterminate']
# type_options = ['routine outpatient', 'routine inpatient', '']
# state = toggle_multiple('Patient State', state_options)

buttons[0]['widget'] = toggle_single('EEG Type', buttons[0])
buttons[1]['widget'] = toggle_multiple('Patient State', buttons[1])
widgets.VBox([buttons[0]['widget'],buttons[1]['widget'], out])



In [ ]:
choices

{'state':    awake  asleep  encephalopathic  sedated  comatose  indeterminate
 0  False   False            False    False     False          False,
 'type':    routine outpatient  routine inpatient  ...  urgent PICU  urgent CICU
 0               False              False  ...        False        False
 
 [1 rows x 7 columns]}